# Minimum probability flow

In [2]:
import numpy as np
import theano.tensor as T

## Preparation
To get started we first have to prepare the data to work with. The steps to synthesis the data are as follows:
1. Form a $n \times n$ matrix $W$ that is symmetric with diagonal entries set to zeros.
2. The bias vector is $n \times 1$ that is either set to zero or takes binary inputs.
3. Initialise a vector of size $n$ that has binary entries, $x^{(1)}$, which is used to generate the subsequent sample values.
4. Given $x^{(i)}$, for each row $j$
\begin{align*}
p_{j}^{(i+1)} &= \sigma\left(\sum_{k=1}^{n}w_{jk}x_{k}^{(i)}\right)\\
%x_{j}^{(i+1)}&=\sigma(\tilde{x}_{j}^{(i+1)})
\end{align*}
which produces a row of values between 0 and 1. Each entry of $x_j^{(i+1)}$ is Bernoulli distributed with parameter $p_{j}^{(i+1)}$.


In [113]:
# Set dimension of the data vector
n = 16

# Initialize weight matrix that is symmetric and has zero diagonal entries
W = np.triu(np.random.normal(0,1,(n,n))).dot(1 - np.eye(n))

# Bias vector with binary inputs
#b = np.random.randint(2, size = n)
b = np.zeros((1,n))

# Seed data vector to generate data
x = np.random.randint(2, size = n)

In [114]:
def sigmoid(x):
    return 1/ (1 + np.exp(-x))

In [115]:
def gendata(x, W, b):
    #print (sigmoid(W.dot(x)+b))
    return np.random.binomial(1,sigmoid(W.dot(x)+b))    

In [117]:
for i in np.arange(10):
    y = gendata(x, W, b)
    print (y)

[[1 0 1 1 1 0 0 0 1 1 1 1 1 1 0 0]]
[[1 0 1 1 1 0 0 0 1 1 1 1 1 1 0 0]]
[[1 0 1 1 1 0 0 0 1 1 1 1 1 1 0 0]]
[[1 0 1 1 1 0 0 0 1 1 1 1 1 1 0 0]]
[[1 0 1 1 1 0 0 0 1 1 1 1 1 1 0 0]]
[[1 0 1 1 1 0 0 0 1 1 1 1 1 1 0 0]]
[[1 0 1 1 1 0 0 0 1 1 1 1 1 1 0 0]]
[[1 0 1 1 1 0 0 0 1 1 1 1 1 1 0 0]]
[[1 0 1 1 1 0 0 0 1 1 1 1 1 1 0 0]]
[[1 0 1 1 1 0 0 0 1 1 1 1 1 1 0 0]]
